# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [82]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,berlevag,70.8578,29.0864,270.12,63,75,10.29,NO,1679347761
1,1,kabala,9.5833,-11.5500,300.81,41,33,4.24,SL,1679347762
2,2,kayes,14.0000,-11.0000,305.86,9,0,2.89,ML,1679347763
3,3,lompoc,34.6391,-120.4579,290.25,76,100,4.92,US,1679347764
4,4,bekescsaba,46.6833,21.1000,283.12,70,100,1.34,HU,1679347765
...,...,...,...,...,...,...,...,...,...,...
544,544,taiyuan,37.8694,112.5603,278.07,75,13,1.00,CN,1679347980
545,545,salalah,17.0151,54.0924,300.20,89,75,2.57,OM,1679348171
546,546,strezhevoy,60.7333,77.5889,266.26,96,96,3.18,RU,1679348172
547,547,guantanamo,20.1444,-75.2092,301.91,55,70,2.76,CU,1679348173


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [97]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)


# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df["Max Temp"] < 275) & (city_data_df["Humidity"] < 60)]

# Drop any rows with null values
ideal_df.dropna()
# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,ekibastuz,51.6667,75.3667,266.87,56,80,3.80,KZ,1679347766
10,10,yellowknife,62.4560,-114.3525,264.44,45,20,3.60,CA,1679347233
69,69,thompson,55.7435,-97.8558,262.24,52,20,1.54,CA,1679347803
183,183,emba,48.8264,58.1442,271.97,55,53,3.01,KZ,1679347902
190,190,sorland,67.6670,12.6934,274.07,58,100,13.86,NO,1679347908
214,214,la ronge,55.1001,-105.2842,268.27,45,75,7.20,CA,1679347923
224,224,ilulissat,69.2167,-51.1000,270.16,59,40,1.03,GL,1679347400
404,404,baruun-urt,46.6806,113.2792,273.67,35,100,2.53,MN,1679348065
482,482,storslett,69.7678,21.0247,270.15,59,79,4.63,NO,1679348126
491,491,zhezkazgan,47.8043,67.7144,268.88,55,6,3.85,KZ,1679348132


### Step 3: Create a new DataFrame called `hotel_df`.

In [57]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City", "Lat", "Lng", "Country", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df

,City,Lat,Lng,Country,Humidity,Hotel Name
7,ekibastuz,51.6667,75.3667,KZ,56,
10,yellowknife,62.4560,-114.3525,CA,45,
69,thompson,55.7435,-97.8558,CA,52,
183,emba,48.8264,58.1442,KZ,55,
190,sorland,67.6670,12.6934,NO,58,
214,la ronge,55.1001,-105.2842,CA,45,
224,ilulissat,69.2167,-51.1000,GL,59,
404,baruun-urt,46.6806,113.2792,MN,35,
482,storslett,69.7678,21.0247,NO,59,
491,zhezkazgan,47.8043,67.7144,KZ,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000

params = {
    
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key, 
    "limit": 12
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
#     # Convert the API response to JSON format
#     name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ekibastuz - nearest hotel: General
yellowknife - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
emba - nearest hotel: No hotel found
sorland - nearest hotel: Fagmek Resorts
la ronge - nearest hotel: La Ronge Hotel and Suites
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
baruun-urt - nearest hotel: No hotel found
storslett - nearest hotel: Reisafjord hotel
zhezkazgan - nearest hotel: KENGIR
flin flon - nearest hotel: Copperbelt Hotel
hami - nearest hotel: Хами


,City,Lat,Lng,Country,Humidity,Hotel Name
7,ekibastuz,51.6667,75.3667,KZ,56,General
10,yellowknife,62.4560,-114.3525,CA,45,No hotel found
69,thompson,55.7435,-97.8558,CA,52,Thompson Inn
183,emba,48.8264,58.1442,KZ,55,No hotel found
190,sorland,67.6670,12.6934,NO,58,Fagmek Resorts
214,la ronge,55.1001,-105.2842,CA,45,La Ronge Hotel and Suites
224,ilulissat,69.2167,-51.1000,GL,59,Best Western Plus Hotel Ilulissat
404,baruun-urt,46.6806,113.2792,MN,35,No hotel found
482,storslett,69.7678,21.0247,NO,59,Reisafjord hotel
491,zhezkazgan,47.8043,67.7144,KZ,55,KENGIR


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [95]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)